In [1]:
import pandas as pd
import numpy as np


### Ознакомимся с таблицами

In [2]:
# Откроем таблицу Звонки, переименуем колонки для удобства и удалим столбец "Id Звонка" 
df_calls = pd.read_excel('Задание для аналитика в отдел аналитики (1).xlsx', sheet_name = '5.3. Звонки')

df_calls = df_calls.rename(columns={'ID Cотрудника':'hr_id','Дата звонка':'date', 'Клиент':'client_id'})
df_calls = df_calls.drop('ID Звонка', axis=1)
df_calls.tail()

,hr_id,Длительность звонка (сек),date,client_id
12746,9845,253,2023-06-28,306187
12747,9503,134,2023-06-28,306188
12748,10450,293,2023-06-28,306191
12749,9392,4,2023-06-28,306192
12750,9392,0,2023-06-28,306192


In [3]:
# Откроем таблицу "Список HR", переименуем колонки для удобства
df_ch = pd.read_excel('Задание для аналитика в отдел аналитики (1).xlsx', sheet_name = '5.1. Список HR')
df_ch = df_ch.rename(columns={'HR':'hr_id','Город':'city'})
df_ch.head()

,hr_id,city
0,9346,Москва
1,9392,Санкт-Петербург
2,9427,Санкт-Петербург
3,9503,Санкт-Петербург
4,9845,Москва


In [4]:
# Откроем таблицу "Планы HR", переименуем колонки для удобства
df_plan = pd.read_excel('Задание для аналитика в отдел аналитики (1).xlsx', sheet_name = '5.2. Планы HR')
df_plan = df_plan.rename(columns={'ID HR':'hr_id','Дата Плана':'date', 'План на уникальные звонки > 3 минут':'plan'})
df_plan

,date,hr_id,plan
0,2023-06-01,9346,3
1,2023-06-02,9346,3
2,2023-06-03,9346,3
3,2023-06-04,9346,3
4,2023-06-05,9346,3
...,...,...,...
535,2023-06-26,9546,3
536,2023-06-27,9546,3
537,2023-06-28,9546,3
538,2023-06-29,9546,3


### Обработаем данные по времени

In [5]:
# Ограничим период анализа
df_calls = df_calls.loc[df_calls['date'] >= '2023-06-01'].sort_values(by='date')


In [6]:
# То же самое сделаем для плана
df_plan = df_plan.loc[(df_plan['date'] >= '2023-06-01') & (df_plan['date'] <= '2023-06-28') ].sort_values(by='date')
df_plan.tail()

,date,hr_id,plan
387,2023-06-28,15345,3
417,2023-06-28,15365,3
447,2023-06-28,9268,3
237,2023-06-28,10153,3
537,2023-06-28,9546,3


### Объединение и преобразвание данных

In [7]:
# Начнем объединение
df = df_calls.merge(df_ch, on='hr_id', how='left')
df.head()

,hr_id,Длительность звонка (сек),date,client_id,city
0,9346,0,2023-06-01,266542,Москва
1,10443,13,2023-06-01,302541,Москва
2,9427,0,2023-06-01,302536,Санкт-Петербург
3,9427,0,2023-06-01,302536,Санкт-Петербург
4,9427,0,2023-06-01,302536,Санкт-Петербург


In [8]:
# удалим звонки длительностью 0 секкунд
df = df.loc[df['Длительность звонка (сек)']!=0]



In [9]:
# перевеедм секунды в минуты и переименуем столбец
df['Длительность звонка (сек)'] = df['Длительность звонка (сек)'] / 60
df = df.rename(columns={'Длительность звонка (сек)':'Длительность (мин)'})
df.head()


,hr_id,Длительность (мин),date,client_id,city
1,10443,0.216667,2023-06-01,302541,Москва
8,10443,0.416667,2023-06-01,302527,Москва
9,10432,8.000000,2023-06-01,302526,Москва
16,10453,5.166667,2023-06-01,302524,Москва
17,9427,0.033333,2023-06-01,227559,Санкт-Петербург


In [10]:
# Отсортируем таблицу по дате, сотруднику отдеа кадров и клиенту
df = df.sort_values(['date', 'hr_id', 'client_id'])
df = df[['date','hr_id','client_id', 'Длительность (мин)', 'city']]
df.head(10)


,date,hr_id,client_id,Длительность (мин),city
269,2023-06-01,9346,202401,0.050000,Москва
270,2023-06-01,9346,202401,0.033333,Москва
271,2023-06-01,9346,202401,0.033333,Москва
272,2023-06-01,9346,202401,0.033333,Москва
274,2023-06-01,9346,202401,0.033333,Москва
286,2023-06-01,9346,202401,0.050000,Москва
104,2023-06-01,9346,210548,0.033333,Москва
105,2023-06-01,9346,210548,0.066667,Москва
380,2023-06-01,9346,230677,0.650000,Москва
268,2023-06-01,9346,250220,0.033333,Москва


In [11]:
# Добавим столбец в котором будет указана сумма времени звонков 1 клиенту от 1 hr в рамках 1 дня
df['mins_per_client'] = df.groupby(['date', 'hr_id','client_id'])['Длительность (мин)'].transform('sum')
df.head(20)
        

,date,hr_id,client_id,Длительность (мин),city,mins_per_client
269,2023-06-01,9346,202401,0.050000,Москва,0.233333
270,2023-06-01,9346,202401,0.033333,Москва,0.233333
271,2023-06-01,9346,202401,0.033333,Москва,0.233333
272,2023-06-01,9346,202401,0.033333,Москва,0.233333
274,2023-06-01,9346,202401,0.033333,Москва,0.233333
286,2023-06-01,9346,202401,0.050000,Москва,0.233333
104,2023-06-01,9346,210548,0.033333,Москва,0.100000
105,2023-06-01,9346,210548,0.066667,Москва,0.100000
380,2023-06-01,9346,230677,0.650000,Москва,0.650000
268,2023-06-01,9346,250220,0.033333,Москва,0.033333


In [12]:
# Удалим строки, в которых сумма звонков однуму клиенту была меньше 3 минут
df = df.loc[df['mins_per_client']>=3]
df.head(10)

,date,hr_id,client_id,Длительность (мин),city,mins_per_client
188,2023-06-01,9346,279592,8.066667,Москва,8.066667
30,2023-06-01,9346,302545,8.300000,Москва,8.300000
151,2023-06-01,9346,302660,11.566667,Москва,11.566667
371,2023-06-01,9392,299976,7.883333,Санкт-Петербург,7.883333
58,2023-06-01,9392,302551,21.766667,Санкт-Петербург,21.833333
59,2023-06-01,9392,302551,0.066667,Санкт-Петербург,21.833333
333,2023-06-01,9427,234919,0.916667,Санкт-Петербург,6.133333
343,2023-06-01,9427,234919,5.216667,Санкт-Петербург,6.133333
61,2023-06-01,9427,302548,6.350000,Санкт-Петербург,6.350000
52,2023-06-01,9427,302554,5.033333,Санкт-Петербург,5.033333


In [13]:
# На этом этапе уберем повторяющихся клиентов (план более 3 минут уже выполнен)
df = df.drop_duplicates(subset='client_id')

In [14]:
# Добавим столбец в котором будет указана количество хороших (плановых) звонков от 1 hr в рамках 1 дня
df['good_calls'] = df.groupby(['date', 'hr_id'])['hr_id'].transform('count')
df.head(20)

,date,hr_id,client_id,Длительность (мин),city,mins_per_client,good_calls
188,2023-06-01,9346,279592,8.066667,Москва,8.066667,3
30,2023-06-01,9346,302545,8.300000,Москва,8.300000,3
151,2023-06-01,9346,302660,11.566667,Москва,11.566667,3
371,2023-06-01,9392,299976,7.883333,Санкт-Петербург,7.883333,2
58,2023-06-01,9392,302551,21.766667,Санкт-Петербург,21.833333,2
333,2023-06-01,9427,234919,0.916667,Санкт-Петербург,6.133333,3
61,2023-06-01,9427,302548,6.350000,Санкт-Петербург,6.350000,3
52,2023-06-01,9427,302554,5.033333,Санкт-Петербург,5.033333,3
318,2023-06-01,9503,249768,2.866667,Санкт-Петербург,3.266667,2
468,2023-06-01,9503,259277,0.433333,Санкт-Петербург,7.983333,2


In [15]:
# заведем новую таблицу с фактичесикими звонками
df_fact = df[['date', 'hr_id','good_calls', 'city']]
df_fact.head(20)

,date,hr_id,good_calls,city
188,2023-06-01,9346,3,Москва
30,2023-06-01,9346,3,Москва
151,2023-06-01,9346,3,Москва
371,2023-06-01,9392,2,Санкт-Петербург
58,2023-06-01,9392,2,Санкт-Петербург
333,2023-06-01,9427,3,Санкт-Петербург
61,2023-06-01,9427,3,Санкт-Петербург
52,2023-06-01,9427,3,Санкт-Петербург
318,2023-06-01,9503,2,Санкт-Петербург
468,2023-06-01,9503,2,Санкт-Петербург


In [16]:
# уберем получившиеся дубликаты 
df_fact= df_fact.drop_duplicates()
df_fact.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 282 entries, 188 to 8632
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        282 non-null    datetime64[ns]
 1   hr_id       282 non-null    int64         
 2   good_calls  282 non-null    int64         
 3   city        244 non-null    object        
dtypes: datetime64[ns](1), int64(2), object(1)
memory usage: 11.0+ KB


In [17]:
df_fact.loc[df_fact['city'].isna()]['hr_id'].value_counts()

14100    15
6699     11
15185     9
8716      3
Name: hr_id, dtype: int64

In [18]:
# Объединим таблицы План и Факт по столбцам даты и Id сотрудника
df_outcome = pd.merge(df_plan, df_fact, how = 'left', on = ['date', 'hr_id'])
df_outcome

,date,hr_id,plan,good_calls,city
0,2023-06-01,9346,3,3.0,Москва
1,2023-06-01,10432,3,4.0,Москва
2,2023-06-01,9268,3,NaN,NaN
3,2023-06-01,9427,3,3.0,Санкт-Петербург
4,2023-06-01,15365,3,NaN,NaN
...,...,...,...,...,...
499,2023-06-28,15345,3,1.0,Москва
500,2023-06-28,15365,3,NaN,NaN
501,2023-06-28,9268,3,NaN,NaN
502,2023-06-28,10153,3,NaN,NaN


In [19]:
# Уберем client_id
df_outcome = df_outcome[['date', 'hr_id', 'plan', 'good_calls', 'city']]
df_outcome

,date,hr_id,plan,good_calls,city
0,2023-06-01,9346,3,3.0,Москва
1,2023-06-01,10432,3,4.0,Москва
2,2023-06-01,9268,3,NaN,NaN
3,2023-06-01,9427,3,3.0,Санкт-Петербург
4,2023-06-01,15365,3,NaN,NaN
...,...,...,...,...,...
499,2023-06-28,15345,3,1.0,Москва
500,2023-06-28,15365,3,NaN,NaN
501,2023-06-28,9268,3,NaN,NaN
502,2023-06-28,10153,3,NaN,NaN


In [20]:
# проверим на пропуски и уберем дубликаты
df_outcome = df_outcome.dropna()
df_outcome = df_outcome.drop_duplicates()
df_outcome.head(10)

,date,hr_id,plan,good_calls,city
0,2023-06-01,9346,3,3.0,Москва
1,2023-06-01,10432,3,4.0,Москва
3,2023-06-01,9427,3,3.0,Санкт-Петербург
5,2023-06-01,9503,3,2.0,Санкт-Петербург
7,2023-06-01,9392,4,2.0,Санкт-Петербург
9,2023-06-01,10443,3,4.0,Москва
10,2023-06-01,10453,3,2.0,Москва
11,2023-06-01,9546,10,5.0,Москва
12,2023-06-01,10700,3,4.0,Санкт-Петербург
13,2023-06-01,9845,2,4.0,Москва


In [21]:
# посмотрим в целом на таблицу
df_outcome.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 244 entries, 0 to 499
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        244 non-null    datetime64[ns]
 1   hr_id       244 non-null    int64         
 2   plan        244 non-null    int64         
 3   good_calls  244 non-null    float64       
 4   city        244 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(1)
memory usage: 11.4+ KB


In [22]:
# посчитаем количество вхождений
df_outcome['city'].value_counts()

Санкт-Петербург    127
Москва             117
Name: city, dtype: int64

In [23]:
# посчитаем неучтенных сотрудников (их данных нет в таблице "Список HR")
df_fact.loc[df_fact['city'].isna()]['hr_id'].value_counts()

14100    15
6699     11
15185     9
8716      3
Name: hr_id, dtype: int64

### Выводы:

1. Всего план выполнили в 244 раза (сотрудник за 1 день сделал более 3 необходимых звонков) из 504, то есть чуть меньше половины.
2. В Петербурге выполнили 127 раз, а в Москве 117. 
3. Важно понимать, что сотрудники без определенного города могут и переломить ситцацию и вывести Москву вперед. Нужно уточнение по принадлежности сотрудников. 